In [ ]:
# Load pre-trained BERT model
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Step 1: Freeze all layers except the last one (classification head)
for param in model.base_model.parameters():
    param.requires_grad = False

# If you'd like to fine-tune additional layers (e.g., the last 2 layers), you can unfreeze those layers as well
for param in model.base_model.encoder.layer[-2:].parameters():
    param.requires_grad = True

In [ ]:
# Load and prepare dataset (example using a sample dataset)
from datasets import load_dataset
from transformers import BertTokenizer

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load a sample dataset (you can replace this with your own dataset)
# For this example, we'll use a sentiment analysis dataset
dataset = load_dataset("glue", "sst2")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Split into train, validation, and test sets
train_data = tokenized_dataset["train"].shuffle(seed=42).select(range(1000))  # Using subset for faster training
val_data = tokenized_dataset["validation"].shuffle(seed=42).select(range(200))
test_data = tokenized_dataset["validation"].shuffle(seed=42).select(range(200, 400))

In [ ]:
from transformers import Trainer, TrainingArguments

# Step 1: Set training arguments for fine-tuning the model
training_args = TrainingArguments(
    output_dir='./results',             # Directory where results will be stored
    num_train_epochs=3,                 # Number of epochs (full passes through the dataset)
    per_device_train_batch_size=16,     # Batch size per GPU/CPU during training
    eval_strategy="epoch",              # Fixed: Changed from evaluation_strategy to eval_strategy
)

# Step 2: Fine-tune only the final classification head (since earlier layers were frozen)
trainer = Trainer(
    model=model,                        # Pre-trained BERT model with frozen layers
    args=training_args,                 # Training arguments
    train_dataset=train_data,           # Training data for fine-tuning
    eval_dataset=val_data,              # Validation data to evaluate performance during training
)

# Step 3: Train the model using PEFT (this performs PEFT because layers were frozen in Step 1)
trainer.train()

In [ ]:
# Evaluate the model
results = trainer.evaluate(eval_dataset=test_data)
print(f"Evaluation results: {results}")
# Note: The metric key depends on what metrics are computed during training
# Common keys include 'eval_loss', but 'eval_accuracy' requires adding accuracy metric to the trainer

In [ ]:
# Example of adjusting learning rate for PEFT optimization
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=5e-5,  # Experiment with different learning rates
    num_train_epochs=5,
    per_device_train_batch_size=16,
)